In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
def split_data(df):
  # Define the proportion of data for each set
  test_size = 0.2  # 20% of the data for testing
  val_size = 0.2   # 20% of the data for validation
  train_size = 1 - (test_size + val_size)  # Remaining data for training

  # Split the data into train-validation-test sets
  train_val_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

  # Calculate the adjusted validation size based on the remaining data after test split
  adjusted_val_size = val_size / (train_size + val_size - test_size)

  # Split the train-validation set into train and validation sets
  train_df, val_df = train_test_split(train_val_df, test_size=adjusted_val_size, random_state=42)

  train_df = pd.DataFrame(data=train_df)
  val_df = pd.DataFrame(data=val_df)
  test_df = pd.DataFrame(data=test_df)

  train_df = train_df.dropna()
  test_df = test_df.dropna()
  val_df = val_df.dropna()

  return train_df, test_df, val_df

In [ ]:
def experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, n_est, maxDep):

  # Separate the features and the target
  X_train = train_df[feature_columns]
  y_train = train_df[target_column]
  # Validate the model with the validation df
  X_val = val_df[feature_columns]
  y_val = val_df[target_column]

  # Create an instance of the Random Forest model
  model = RandomForestRegressor(n_estimators=n_est, random_state=42,
                      criterion='squared_error',max_depth = maxDep)
  # Train the linear model
  model.fit(X_train, y_train)

  y_val_pred = model.predict(X_val)

  # Calculate evaluation metrics
  mse_validation = mean_squared_error(y_val, y_val_pred)


  X_test = test_df[feature_columns]
  y_test = test_df[target_column]

  y_test_pred = model.predict(X_test)

  # Calculate evaluation metrics
  mse_test = mean_squared_error(y_test, y_test_pred)

  # Print the evaluation metrics
  return mse_validation, mse_test


In [ ]:
def get_best_params(train_df, test_df, val_df, target_column, feature_columns):
  mse_base = experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 10, 1)
  for n_est in [25, 28, 31, 34, 37, 40]:
    for maxDep in range(5, 15):
      mse_val = experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, n_est, maxDep)[0]
      if mse < mse_base:
        best_n_est = n_est
        best_maxDep = maxDep
        mse_base = mse
  print("n_est: "+ str(best_n_est) + ", maxDep: " + str(best_maxDep) + ": " +str(mse_base))



### **Training Set including Rain and Holiday**

In [ ]:
data_training_set = pd.read_csv('/content/drive/My Drive/Bicing_dfs/TrainingSet.csv')

In [ ]:
data_training_set

,station_id,year,month,day,hour,occupation,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday
0,405,2022,6,23,23,0.911397,0.989130,0.969862,0.974308,0.971014,0.0,False
1,75,2022,6,14,16,0.976852,0.814815,0.851035,0.689815,0.738290,0.0,False
2,418,2022,6,20,2,0.666667,0.688272,0.660494,0.677707,0.779321,0.0,False
3,110,2022,6,22,13,0.796627,0.868849,0.964087,0.968254,0.980159,0.0,False
4,61,2022,6,23,11,0.525306,0.171569,0.236826,0.345588,0.500000,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8796999,150,2021,1,6,3,0.653846,0.653846,0.486923,0.283077,0.320000,0.0,True
8797000,175,2021,1,14,23,0.333333,0.333333,0.333333,0.358025,0.376543,0.0,False
8797001,335,2021,1,7,10,0.216049,0.219136,0.293210,0.265432,0.243827,0.0,False
8797002,514,2021,1,16,6,0.911765,0.941176,0.906863,0.882353,0.916360,0.0,False


In [ ]:
data_training_set.shape

(8797004, 12)

## **Merging Geospatial Station Data**

In [ ]:
data_stations = pd.read_csv('/content/drive/My Drive/Bicing_dfs/data_stations.csv')

In [ ]:
data_stations.shape

(14723891, 10)

In [ ]:
data_stations = data_stations[data_stations['year'].isin([2021, 2022])]

In [ ]:
merge_df = pd.merge(data_training_set, data_stations,how="left", on=['station_id', 'day','month','year','hour'])

In [ ]:
merge_df

,station_id,year,month,day,hour,occupation,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,post_code,capacity
0,405,2022,6,23,23,0.911397,0.989130,0.969862,0.974308,0.971014,0.0,False,41.385400,2.152180,28.0,8029.0,26.0
1,75,2022,6,14,16,0.976852,0.814815,0.851035,0.689815,0.738290,0.0,False,41.385004,2.142911,40.0,8029.0,21.0
2,418,2022,6,20,2,0.666667,0.688272,0.660494,0.677707,0.779321,0.0,False,41.391062,2.180114,10.0,8018.0,27.0
3,110,2022,6,22,13,0.796627,0.868849,0.964087,0.968254,0.980159,0.0,False,41.385258,2.155089,25.0,8011.0,21.0
4,61,2022,6,23,11,0.525306,0.171569,0.236826,0.345588,0.500000,0.0,False,41.392387,2.167935,26.0,8009.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8796999,150,2021,1,6,3,0.653846,0.653846,0.486923,0.283077,0.320000,0.0,True,41.406591,2.203028,4.0,8005.0,27.0
8797000,175,2021,1,14,23,0.333333,0.333333,0.333333,0.358025,0.376543,0.0,False,41.406529,2.209122,4.0,8019.0,27.0
8797001,335,2021,1,7,10,0.216049,0.219136,0.293210,0.265432,0.243827,0.0,False,41.393535,2.123123,92.0,8034.0,27.0
8797002,514,2021,1,16,6,0.911765,0.941176,0.906863,0.882353,0.916360,0.0,False,41.409507,2.188506,16.0,8018.0,18.0


## **Filling geospatial NaN's with station_id means**

In [ ]:
# Check for NaN values in each column
nan_columns = merge_df.columns[merge_df.isna().any()].tolist()

# Filter rows with NaN values in any NaN column
rows_with_nan = merge_df[merge_df[nan_columns].isna().any(axis=1)]

rows_with_nan.shape

(376722, 17)

In [ ]:
rows_with_nan

,station_id,year,month,day,hour,occupation,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,post_code,capacity
30307,160,2022,6,14,7,0.666667,1.000000,1.000000,1.000000,1.000000,0.0,False,NaN,NaN,NaN,NaN,NaN
31640,55,2022,6,19,22,0.947368,0.900000,0.297619,0.119048,0.017857,0.0,False,NaN,NaN,NaN,NaN,NaN
120523,214,2022,6,24,17,0.600000,0.565000,0.556667,0.551806,0.506667,0.0,True,NaN,NaN,NaN,NaN,NaN
253967,144,2022,6,5,17,0.520000,0.421667,0.399861,0.642222,0.678889,0.0,False,NaN,NaN,NaN,NaN,NaN
293702,163,2022,6,17,17,0.000000,0.638889,0.731884,0.868934,0.930556,0.0,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8579755,231,2021,1,5,18,0.521739,0.317763,0.905303,0.912879,0.890975,0.0,False,NaN,NaN,NaN,NaN,NaN
8603224,491,2021,1,10,8,0.333333,0.346154,0.346154,0.346154,0.346154,0.0,False,NaN,NaN,NaN,NaN,NaN
8666882,294,2021,1,25,9,0.786111,0.766563,0.708333,0.649306,0.666667,0.0,False,NaN,NaN,NaN,NaN,NaN
8727229,433,2021,1,3,12,0.433333,0.465517,0.482759,0.545977,0.646552,0.0,False,NaN,NaN,NaN,NaN,NaN


In [ ]:
merge_df_cleaned = merge_df.dropna()

after_indexes = merge_df_cleaned.index
before_indexes = merge_df.index
dropped_indexes = before_indexes.difference(after_indexes)


In [ ]:
station_means = pd.DataFrame()
station_means['station_id']= merge_df.loc[dropped_indexes]['station_id'].unique()
station_means.set_index('station_id', inplace=True)


In [ ]:
for id in merge_df.loc[dropped_indexes]['station_id'].unique():
  station_means.loc[id, 'lat']=data_stations[data_stations['station_id']==id]['lat'].mean()
  station_means.loc[id, 'lon']=data_stations[data_stations['station_id']==id]['lon'].mean()
  station_means.loc[id, 'altitude']=data_stations[data_stations['station_id']==id]['altitude'].mean()
  station_means.loc[id, 'capacity']=data_stations[data_stations['station_id']==id]['capacity'].mean()

In [ ]:
station_means

,lat,lon,altitude,capacity
station_id,,,,
160,41.410976,2.219372,4.0,32.994647
55,41.381398,2.173313,8.0,18.893651
214,41.395203,2.133487,72.0,25.000000
144,41.405627,2.197712,4.0,26.772416
163,41.394211,2.200876,3.0,26.811438
...,...,...,...,...
508,41.409092,2.165638,70.0,26.954764
277,41.408211,2.168996,60.0,26.949251
480,41.433326,2.162926,92.0,28.000000


In [ ]:
for index in dropped_indexes:
  station_id = merge_df.loc[index]['station_id']

  merge_df.loc[index, 'lat'] = station_means.loc[station_id]['lat']
  merge_df.loc[index, 'lon'] = station_means.loc[station_id]['lon']
  merge_df.loc[index, 'altitude'] = station_means.loc[station_id]['altitude']
  merge_df.loc[index, 'capacity'] = station_means.loc[station_id]['capacity']


In [ ]:
merge_df.loc[dropped_indexes]

,station_id,year,month,day,hour,occupation,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,post_code,capacity
30307,160,2022,6,14,7,0.666667,1.000000,1.000000,1.000000,1.000000,0.0,False,41.410976,2.219372,4.0,NaN,32.994647
31640,55,2022,6,19,22,0.947368,0.900000,0.297619,0.119048,0.017857,0.0,False,41.381398,2.173313,8.0,NaN,18.893651
120523,214,2022,6,24,17,0.600000,0.565000,0.556667,0.551806,0.506667,0.0,True,41.395203,2.133487,72.0,NaN,25.000000
253967,144,2022,6,5,17,0.520000,0.421667,0.399861,0.642222,0.678889,0.0,False,41.405627,2.197712,4.0,NaN,26.772416
293702,163,2022,6,17,17,0.000000,0.638889,0.731884,0.868934,0.930556,0.0,False,41.394211,2.200876,3.0,NaN,26.811438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8579755,231,2021,1,5,18,0.521739,0.317763,0.905303,0.912879,0.890975,0.0,False,41.407772,2.163665,65.0,NaN,24.000000
8603224,491,2021,1,10,8,0.333333,0.346154,0.346154,0.346154,0.346154,0.0,False,41.430669,2.145645,125.0,NaN,26.973487
8666882,294,2021,1,25,9,0.786111,0.766563,0.708333,0.649306,0.666667,0.0,False,41.436347,2.170675,72.0,NaN,24.000000
8727229,433,2021,1,3,12,0.433333,0.465517,0.482759,0.545977,0.646552,0.0,False,41.386226,2.116861,70.0,NaN,29.484088


In [ ]:
final_data = merge_df

In [ ]:
# Combine the year, month, and day columns into a single 'date' column
final_data['date'] = pd.to_datetime(final_data[['year', 'month', 'day']])

# Extract the day of the week from the 'date' column
final_data['day_of_week'] = final_data['date'].dt.day_name()

In [ ]:
final_data['day_of_week']

0           Thursday
1            Tuesday
2             Monday
3          Wednesday
4           Thursday
             ...    
8796999    Wednesday
8797000     Thursday
8797001     Thursday
8797002     Saturday
8797003       Monday
Name: day_of_week, Length: 8797004, dtype: object

In [ ]:
def assign_day_category(row):
    if row == 'Monday':
        return 0
    if row == 'Tuesday':
        return 0
    if row == 'Wednesday':
        return 0
    if row == 'Thursday':
        return 0
    if row == 'Friday':
        return 1
    if row == 'Saturday':
        return 2
    if row == 'Sunday':
        return 3

final_data['day_category'] = final_data['day_of_week'].apply(assign_day_category)


In [ ]:
def reassign_day_of_week(row):
    if row == 'Monday':
        return 0
    if row == 'Tuesday':
        return 1
    if row == 'Wednesday':
        return 2
    if row == 'Thursday':
        return 3
    if row == 'Friday':
        return 4
    if row == 'Saturday':
        return 5
    if row == 'Sunday':
        return 6

final_data['day_of_week'] = final_data['day_of_week'].apply(reassign_day_of_week)


In [ ]:
final_data['day_category'].unique()

array([0, 3, 2, 1])

In [ ]:
final_data

station_id                       int64
year                             int64
month                            int64
day                              int64
hour                             int64
occupation                     float64
occupation_1h_before           float64
occupation_2h_before           float64
occupation_3h_before           float64
occupation_4h_before           float64
VALOR_LECTURA                  float64
holiday                           bool
lat                            float64
lon                            float64
altitude                       float64
post_code                      float64
capacity                       float64
date                    datetime64[ns]
day_of_week                      int64
day_category                     int64
dtype: object

In [ ]:
final_data.to_csv("final_data.csv",index=False)
files.download('final_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Experimentation

In [ ]:
train_df, test_df, val_df = split_data(final_data)

In [ ]:
final_data.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'occupation',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'post_code', 'capacity', 'date', 'day_of_week',
       'day_category'],
      dtype='object')

In [ ]:

#add day_category
target_column = "occupation"
feature_columns = ['station_id', 'year', 'month', 'day', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'day_category']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)


(0.01206230066441968, 0.012046667331511827)

In [ ]:

#add day_of_week instead of day_category
target_column = "occupation"
feature_columns = ['station_id', 'year', 'month', 'day', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'day_of_week']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)


(0.01206243334388021, 0.012047204552704511)

In [ ]:
#adding geospatial columns to the model

target_column = "occupation"
feature_columns = ['station_id', 'year', 'month', 'day', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'capacity', 'day_category']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)


(0.011468035465618817, 0.011446142830840738)

In [ ]:
#adding only altura and capacity

target_column = "occupation"
feature_columns = ['station_id', 'year', 'month', 'day', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday','day_category',
       'altitude', 'capacity']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)

(0.011681867957048276, 0.011669289174675065)

In [ ]:
#my best guess: no year, no day, add geospatial variables and day category

target_column = "occupation"
feature_columns = ['station_id', 'month', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'capacity', 'day_category']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)

##This looks like to be the besr score


(0.011470606446642502, 0.01145109737345054)

In [ ]:
#my best guess 2: no year, no day, no geospatial variables just altitude and day category

target_column = "occupation"
feature_columns = ['station_id', 'month', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday',
       'altitude', 'capacity', 'day_of_week']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)



(0.011685313542447728, 0.011672304379813139)

In [ ]:
#my best guess 2: no year, no day, no geospatial variables, no capacity just altitude and day category

target_column = "occupation"
feature_columns = ['station_id', 'month', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday',
       'altitude', 'day_of_week']


experiment_differentcolumns(train_df, test_df, val_df, target_column, feature_columns, 24, 15)



(0.011734308108011626, 0.011723286385763537)

Model for Kaggle:

In [ ]:
final_data.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'occupation',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'post_code', 'capacity', 'date', 'day_of_week',
       'day_category'],
      dtype='object')

In [ ]:
final_data = final_data.dropna()

In [ ]:

target_column = "occupation"
feature_columns = ['station_id', 'month', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'capacity', 'day_category']

X = final_data[feature_columns]
y = final_data[target_column]

n_est = 500
maxDep = 12

model = RandomForestRegressor(n_estimators=n_est, random_state=42,
                      criterion='squared_error',max_depth = maxDep)
# Train the linear model
model.fit(X, y)

In [ ]:
new_df = pd.read_csv('/content/drive/My Drive/Bicing_dfs/metadata_sample_submission.csv', low_memory=False)

#weekday'i day _category'ye çevir
new_df['year'] = 2023
new_df['date'] = pd.to_datetime(new_df[['year', 'month', 'day']])
new_df['day_of_week'] = new_df['date'].dt.weekday
new_df['day_category'] = new_df['day_of_week'].apply(assign_day_category)

#yağmur ve holidday datasını birleştir
newColumns = pd.read_csv('/content/drive/My Drive/Bicing_dfs/newColumns.csv',low_memory=False)
newColumns = newColumns.rename(columns={"VALOR_LECTURA":"rainPerSqMeter"})

new_df = pd.merge(new_df,newColumns,
                how='left',
                left_on=['day','month','year','hour'],
                right_on=['day','month','year','hour'])

In [ ]:
new_df

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,date,day_of_week,day_category,rainPerSqMeter,holiday
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074,2023,2023-03-07,1,None,0.0,False
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449,2023,2023-03-23,3,None,0.0,False
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852,2023,2023-03-31,4,None,0.0,False
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111,2023,2023-03-23,3,None,0.0,False
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160,2023,2023-03-04,5,None,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54994,54994,269,3,14,3,0.478261,0.478261,0.478261,0.478261,2023,2023-03-14,1,None,0.0,False
54995,54995,280,3,1,5,0.098765,0.074074,0.074074,0.077160,2023,2023-03-01,2,None,0.1,False
54996,54996,180,3,20,16,0.782680,0.821895,0.812092,0.777778,2023,2023-03-20,0,None,0.0,False
54997,54997,277,3,14,11,0.774691,0.953704,0.972222,0.472222,2023,2023-03-14,1,None,0.0,False


In [ ]:
stations2023 = pd.read_csv('/content/drive/My Drive/Bicing_dfs/stations2023.csv', low_memory=False)

In [ ]:
stations2023

,station_id,year,month,day,hour,lat,lon,altitude,post_code,capacity
0,1,2022,12,31,22,41.397978,2.180107,16.0,8013.0,46.0
1,1,2022,12,31,23,41.397978,2.180107,16.0,8013.0,46.0
2,1,2023,1,1,0,41.397978,2.180107,16.0,8013.0,46.0
3,1,2023,1,1,1,41.397978,2.180107,16.0,8013.0,46.0
4,1,2023,1,1,2,41.397978,2.180107,16.0,8013.0,46.0
...,...,...,...,...,...,...,...,...,...,...
1695089,520,2023,5,31,18,41.347695,2.119480,1.0,8908.0,16.0
1695090,520,2023,5,31,19,41.347695,2.119480,1.0,8908.0,16.0
1695091,520,2023,5,31,20,41.347695,2.119480,1.0,8908.0,16.0
1695092,520,2023,5,31,21,41.347695,2.119480,1.0,8908.0,16.0


In [ ]:
new_df = pd.merge(new_df,stations2023,
                how='left',
                left_on=['station_id','day','month','year','hour'],
                right_on=['station_id','day','month','year','hour'])

In [ ]:
def assign_day_category(row):
    if row == 0:
        return 0
    if row == 1:
        return 0
    if row == 2:
        return 0
    if row == 3:
        return 0
    if row == 4:
        return 1
    if row == 5:
        return 2
    if row == 6:
        return 3

new_df['day_category'] = new_df['day_of_week'].apply(assign_day_category)

In [ ]:
new_df

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,date,day_of_week,day_category,rainPerSqMeter,holiday,lat,lon,altitude,post_code,capacity
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074,2023,2023-03-07,1,0,0.0,False,41.387306,2.163126,25.0,8007.0,27.0
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449,2023,2023-03-23,3,0,0.0,False,41.398624,2.120494,117.0,8034.0,23.0
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852,2023,2023-03-31,4,1,0.0,False,41.396839,2.175660,21.0,8013.0,27.0
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111,2023,2023-03-23,3,0,0.0,False,41.405007,2.134603,114.0,8022.0,24.0
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160,2023,2023-03-04,5,2,0.0,False,41.402988,2.134469,106.0,8022.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54994,54994,269,3,14,3,0.478261,0.478261,0.478261,0.478261,2023,2023-03-14,1,0,0.0,False,41.448152,2.192940,26.0,8033.0,23.0
54995,54995,280,3,1,5,0.098765,0.074074,0.074074,0.077160,2023,2023-03-01,2,0,0.1,False,41.413866,2.177727,49.0,8041.0,26.0
54996,54996,180,3,20,16,0.782680,0.821895,0.812092,0.777778,2023,2023-03-20,0,0,0.0,False,41.367557,2.138822,14.0,8014.0,51.0
54997,54997,277,3,14,11,0.774691,0.953704,0.972222,0.472222,2023,2023-03-14,1,0,0.0,False,41.408211,2.168996,60.0,8025.0,27.0


In [ ]:
new_df['occupation_1h_before']=new_df['ctx-1']
new_df['occupation_2h_before']=new_df['ctx-2']
new_df['occupation_3h_before']=new_df['ctx-3']
new_df['occupation_4h_before']=new_df['ctx-4']
new_df['VALOR_LECTURA']=new_df['rainPerSqMeter']


In [ ]:
new_df

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,...,lat,lon,altitude,post_code,capacity,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074,2023,...,41.387306,2.163126,25.0,8007.0,27.0,0.824074,0.799383,0.780864,0.753086,0.0
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449,2023,...,41.398624,2.120494,117.0,8034.0,23.0,0.601449,0.532609,0.536232,0.463768,0.0
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852,2023,...,41.396839,2.175660,21.0,8013.0,27.0,0.601852,0.611111,0.709877,0.787037,0.0
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111,2023,...,41.405007,2.134603,114.0,8022.0,24.0,0.736111,0.819444,0.809028,0.753472,0.0
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160,2023,...,41.402988,2.134469,106.0,8022.0,27.0,0.827160,0.814815,0.802469,0.861111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54994,54994,269,3,14,3,0.478261,0.478261,0.478261,0.478261,2023,...,41.448152,2.192940,26.0,8033.0,23.0,0.478261,0.478261,0.478261,0.478261,0.0
54995,54995,280,3,1,5,0.098765,0.074074,0.074074,0.077160,2023,...,41.413866,2.177727,49.0,8041.0,26.0,0.077160,0.074074,0.074074,0.098765,0.1
54996,54996,180,3,20,16,0.782680,0.821895,0.812092,0.777778,2023,...,41.367557,2.138822,14.0,8014.0,51.0,0.777778,0.812092,0.821895,0.782680,0.0
54997,54997,277,3,14,11,0.774691,0.953704,0.972222,0.472222,2023,...,41.408211,2.168996,60.0,8025.0,27.0,0.472222,0.972222,0.953704,0.774691,0.0


In [ ]:

X = new_df[feature_columns]


In [ ]:
X_cleaned = X.dropna()

after_indexes = X_cleaned.index
before_indexes = X.index
dropped_indexes = before_indexes.difference(after_indexes)


In [ ]:
X.loc[dropped_indexes]

,station_id,month,hour,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,capacity,day_category
7180,269,3,15,0.565217,0.565217,0.568841,0.565217,0.0,False,NaN,NaN,NaN,NaN,0
10896,324,3,23,0.941358,0.833333,0.867284,0.824074,0.0,False,NaN,NaN,NaN,NaN,1
12825,324,3,4,0.830247,0.830247,0.793210,0.787037,0.0,False,NaN,NaN,NaN,NaN,2
13027,400,3,17,0.791667,0.702652,0.270833,0.157197,0.0,False,NaN,NaN,NaN,NaN,0
15205,371,3,10,0.678241,0.768519,0.844907,0.400463,0.0,False,NaN,NaN,NaN,NaN,0
22258,324,3,18,0.777778,0.796296,0.734568,0.611111,0.0,False,NaN,NaN,NaN,NaN,1
24076,324,3,9,0.774691,0.756173,0.793210,0.814815,0.0,False,NaN,NaN,NaN,NaN,2
30053,45,3,17,0.086420,0.240741,0.549383,0.487654,0.0,False,NaN,NaN,NaN,NaN,1
37676,324,3,13,0.203704,0.280864,0.342593,0.330247,0.0,False,NaN,NaN,NaN,NaN,1
44709,70,3,2,0.808642,0.802469,0.154321,0.243827,0.0,False,NaN,NaN,NaN,NaN,0


In [ ]:
for index in dropped_indexes:
  station_id = X.loc[index]['station_id']

  X.loc[index, 'lat'] = stations2023[stations2023['station_id']==station_id]['lat'].mean()
  X.loc[index, 'lon'] = stations2023[stations2023['station_id']==station_id]['lon'].mean()
  X.loc[index, 'altitude'] = stations2023[stations2023['station_id']==station_id]['altitude'].mean()
  X.loc[index, 'capacity'] = stations2023[stations2023['station_id']==station_id]['capacity'].mean()


In [ ]:
X.loc[dropped_indexes]

,station_id,month,hour,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,capacity,day_category
7180,269,3,15,0.565217,0.565217,0.568841,0.565217,0.0,False,41.448152,2.192940,26.0,23.000000,0
10896,324,3,23,0.941358,0.833333,0.867284,0.824074,0.0,False,41.396946,2.136346,80.0,26.846267,1
12825,324,3,4,0.830247,0.830247,0.793210,0.787037,0.0,False,41.396946,2.136346,80.0,26.846267,2
13027,400,3,17,0.791667,0.702652,0.270833,0.157197,0.0,False,41.369451,2.187960,3.0,43.987978,0
15205,371,3,10,0.678241,0.768519,0.844907,0.400463,0.0,False,41.404052,2.181199,19.0,36.000000,0
22258,324,3,18,0.777778,0.796296,0.734568,0.611111,0.0,False,41.396946,2.136346,80.0,26.846267,1
24076,324,3,9,0.774691,0.756173,0.793210,0.814815,0.0,False,41.396946,2.136346,80.0,26.846267,2
30053,45,3,17,0.086420,0.240741,0.549383,0.487654,0.0,False,41.391304,2.189476,5.0,26.934600,1
37676,324,3,13,0.203704,0.280864,0.342593,0.330247,0.0,False,41.396946,2.136346,80.0,26.846267,1
44709,70,3,2,0.808642,0.802469,0.154321,0.243827,0.0,False,41.380393,2.160651,15.0,27.000000,0


In [ ]:
X

,station_id,month,hour,occupation_1h_before,occupation_2h_before,occupation_3h_before,occupation_4h_before,VALOR_LECTURA,holiday,lat,lon,altitude,capacity,day_category
0,394,3,8,0.824074,0.799383,0.780864,0.753086,0.0,False,41.387306,2.163126,25.0,27.0,0
1,337,3,12,0.601449,0.532609,0.536232,0.463768,0.0,False,41.398624,2.120494,117.0,23.0,0
2,368,3,1,0.601852,0.611111,0.709877,0.787037,0.0,False,41.396839,2.175660,21.0,27.0,1
3,327,3,15,0.736111,0.819444,0.809028,0.753472,0.0,False,41.405007,2.134603,114.0,24.0,0
4,328,3,20,0.827160,0.814815,0.802469,0.861111,0.0,False,41.402988,2.134469,106.0,27.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54994,269,3,3,0.478261,0.478261,0.478261,0.478261,0.0,False,41.448152,2.192940,26.0,23.0,0
54995,280,3,5,0.077160,0.074074,0.074074,0.098765,0.1,False,41.413866,2.177727,49.0,26.0,0
54996,180,3,16,0.777778,0.812092,0.821895,0.782680,0.0,False,41.367557,2.138822,14.0,51.0,0
54997,277,3,11,0.472222,0.972222,0.953704,0.774691,0.0,False,41.408211,2.168996,60.0,27.0,0


In [ ]:
y_pred = model.predict(X)

In [ ]:
y_pred

array([0.82517512, 0.66463483, 0.59841241, ..., 0.74720966, 0.56166507,
       0.70178247])

In [ ]:
from google.colab import files

submission = pd.DataFrame(data=y_pred,columns=['percentage_docks_available'])
submission['index'] = submission.index

In [ ]:
submission.to_csv("submission_nans_fillded.csv",index=False)
files.download('submission_nans_fillded.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submission

,percentage_docks_available,index
0,0.827802,0
1,0.668342,1
2,0.592725,2
3,0.734633,3
4,0.840871,4
...,...,...
54982,0.478089,54982
54983,0.194053,54983
54984,0.742998,54984
54985,0.550626,54985


In [ ]:
new_df.columns

NameError: ignored

In [ ]:
feature_columns = ['station_id', 'month', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before', 'VALOR_LECTURA', 'holiday', 'lat', 'lon',
       'altitude', 'capacity', 'day_category']

X = final_data[feature_columns]
y = final_data['occupation']

model_2 = RandomForestRegressor(n_estimators=1125, random_state=42,
                      criterion='squared_error',max_depth = 12)
# Train the linear model
model_2.fit(X, y)